# ClipCard Simulation: Policy/Trust & Safety
**Purpose:** This notebook runs a Monte Carlo simulation to estimate the effectiveness of ClipCard parameters for a policy pilot, such as the rollout of a new classifier.
It simulates the `verified_wrongful_rate_per_10k` over time and checks if it would trigger the `kill_criteria` defined in a ClipCard. This helps quantify the risk of deploying a new policy enforcement mechanism.

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport json# Load example ClipCardtry:    with open('../../examples/policy-ts-test.clipcard.json', 'r') as f:        example_card = json.load(f)    print(f"Loaded example ClipCard: {example_card['id']}")except FileNotFoundError:    print("No example ClipCard found. Using synthetic data. ")    example_card = {        "id": "SIM-POLICY-001",        "kill_criteria": [            {"condition": "verified_wrongful_rate_per_10k >= 0.8", "action": "Pause policy"}        ]    }

### Simulation Parameters
Configure simulation runs for a policy pilot.

In [ ]:
# Simulation configurationN_SIMULATIONS = 1000      # Number of simulated policy pilotsN_TIMESTEPS = 30         # Number of days in the pilotBASE_WRONGFUL_RATE = 0.4 # The expected wrongful rate per 10k actionsRATE_VOLATILITY = 0.1    # How much the rate can fluctuate day-to-dayKILL_THRESHOLD = 0.8     # From the ClipCardprint(f"Running {N_SIMULATIONS} simulations for {N_TIMESTEPS} days each...")print(f"Base Wrongful Rate (per 10k): {BASE_WRONGFUL_RATE}")print(f"Rate Volatility (SD per day): {RATE_VOLATILITY}")print(f"Kill Threshold: >= {KILL_THRESHOLD}")

### Monte Carlo Simulation
Run simulations of the wrongful action rate using a random walk model.

In [ ]:
results = []all_paths = []for i in range(N_SIMULATIONS):    rate_path = [BASE_WRONGFUL_RATE]    triggered = False    for t in range(1, N_TIMESTEPS):        # Random walk model for the rate        next_rate = rate_path[-1] + np.random.normal(0, RATE_VOLATILITY)        if next_rate < 0: next_rate = 0 # Rate can't be negative        rate_path.append(next_rate)        if next_rate >= KILL_THRESHOLD:            triggered = True        all_paths.append(rate_path)    results.append({        'sim_id': i,        'max_rate': max(rate_path),        'triggered': triggered    })df = pd.DataFrame(results)print(f"
Simulation complete!")print(f"Pilots that triggered the kill criterion: {df['triggered'].sum()} ({df['triggered'].mean()*100:.1f}%)")

### Visualization
Plot simulated rate paths and the distribution of maximum rates.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))# Plot a few sample rate pathsfor i in range(min(20, N_SIMULATIONS)):    color = 'red' if max(all_paths[i]) >= KILL_THRESHOLD else 'grey'    alpha = 0.9 if color == 'red' else 0.3    axes[0].plot(all_paths[i], color=color, alpha=alpha)axes[0].axhline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (>= {KILL_THRESHOLD})')axes[0].set_xlabel('Time (days)')axes[0].set_ylabel('Wrongful Rate (per 10k)')axes[0].set_title('Sample Simulated Rate Paths')axes[0].legend()axes[0].set_ylim(bottom=0)# Plot the distribution of maximum rate valuesaxes[1].hist(df['max_rate'], bins=40, alpha=0.7, edgecolor='black')axes[1].axvline(KILL_THRESHOLD, color='red', linestyle='--', label=f'Kill threshold (>= {KILL_THRESHOLD})')axes[1].set_xlabel('Maximum Rate reached during simulation')axes[1].set_ylabel('Frequency (Number of Pilots)')axes[1].set_title('Distribution of Maximum Wrongful Rate')axes[1].legend()plt.tight_layout()plt.show()print(f"
** Interpretation **")print(f"In {N_SIMULATIONS} simulated 30-day pilots:")print(f"- The wrongful action rate kill criterion was triggered {df['triggered'].mean()*100:.1f}% of the time.")print(f"- This simulation helps to calibrate the kill threshold. A lower threshold would catch more excursions but might also trigger more often during normal volatility.")